In [3]:
import pandas as pd
import s3fs
import boto3
import json
from datetime import datetime

In [ ]:
fs = s3fs.S3FileSystem()
s3 = boto3.client('s3')
fnames = fs.ls('s3://streetview-w210/')
fnames = [x for x in fnames if (x.endswith('.json') or x.endswith('.jpg'))]
fnames2 = fs.ls('s3://streetview-w210/gsv')
fnames2 = [x for x in fnames2 if (x.endswith('.json') or x.endswith('.jpg'))]
len(fnames2)

In [29]:
fnames = [x[len('streetview-w210/'):] for x in fnames]
fnames2 = [x[len('streetview-w210/gsv/'):] for x in fnames2]
fnames2 = set(fnames2)

In [32]:
fnames[0]

'10076_45.jpg'

In [33]:
my_list = []
cnt = 0
for name in fnames:
    cnt += 1
    if name in fnames2:
        continue
    else:
        my_list.append(name)
my_list

[]

In [35]:
fs = s3fs.S3FileSystem()
s3 = boto3.client('s3')
fnames = fs.ls('s3://streetview-w210/')
fnames = [x for x in fnames if (x.endswith('.json') or x.endswith('.jpg'))]
len(fnames)

25485

In [7]:
# Move files from outer folder to gsv folder
bucket_name = 'streetview-w210'
s3 = boto3.resource('s3')

start = datetime.now()

for i,fname in enumerate(fnames):
    # Copy to gsv folder
    copy_source = {
        'Bucket': bucket_name,
        'Key': fname[len(bucket_name)+1:]
    }
    try:
        s3.meta.client.copy(copy_source, bucket_name, 'gsv/' + fname[len(bucket_name)+1:])
    except:
        print(fname)


In [36]:
# After we've confirmed that they've all copied over, we can delete them
bucket_name = 'streetview-w210'
s3 = boto3.resource('s3')
start = datetime.now()

for i,fname in enumerate(fnames):
    try:
        s3.Object(bucket_name, fname[len(bucket_name)+1:]).delete()
    except:
        print(fname)

print(datetime.now()-start)

0:08:22.233946


In [4]:
# Now identify all the bad json files
s3_client = boto3.client('s3')
fnames = fs.ls('s3://streetview-w210/gsv/')
fnames = [x for x in fnames if x.endswith('.json')]

bucket_name = 'streetview-w210'
bad_files = []

start = datetime.now()

for i,fname in enumerate(fnames):
    json_path = 's3://{}'.format(fname)
    obj = s3_client.get_object(Bucket=bucket_name, Key=fname[len(bucket_name)+1:])
    json_file = obj['Body'].read().decode('utf-8')
    
    file_data = json.loads(json_file)
    if 'date' not in file_data:
        # Contains bad data. Add to list to remove later
        bad_files.append(fname[len(bucket_name)+1:-5])

print(datetime.now()-start)
print(len(bad_files), bad_files[0])

KeyboardInterrupt: 

In [ ]:
# Create all 5 versions of each file name for each file
suffixes = ['_45.jpg','_135.jpg','_225.jpg','_315.jpg','.json']

all_bad = []

for file in bad_files:
    for suffix in suffixes:
        all_bad.append('streetview-w210/' + file + suffix)
len(all_bad)

In [ ]:
all_bad = set(all_bad)

In [45]:
# Now delete all the bad files and their associated images
prefix = len('streetview-w210/')
# get a list of all bad file names first
fnames = fs.ls('s3://streetview-w210/gsv/')
for x in fnames:
    if x in all_bad:
        try:
            s3.Object(bucket_name, x[prefix:]).delete()
        except:
            print(x)